### 1. Fetch PDB sequences and generate blast DB <br>

Each PDB formatted file includes "SEQRES records" which list the primary sequence of the polymeric molecules present in the entry. This sequence information is also available as a FASTA download

In [ ]:
import pickle
import numpy as np
from pathlib import Path

In [ ]:
import os
import subprocess
import pandas as pd
from Bio.Blast import NCBIXML
from pathlib import Path
import sys 

if not os.path.exists("PDB_blast_db"):
    os.mkdir("PDB_blast_db")

if not os.path.exists("PDB_blast_db/pdb_seqres.txt"):
    subprocess.call("wget http://ftp.wwpdb.org/pub/pdb/derived_data/pdb_seqres.txt.gz",shell=True)    
    subprocess.call("gzip -d pdb_seqres.txt.gz",shell=True)
    subprocess.call("mv pdb_seqres.txt ./PDB_blast_db/",shell=True)
    
if not os.path.exists("PDB_blast_db/pdb_seqres.txt.psq"):
    subprocess.call("makeblastdb -in pdb_seqres.txt -dbtype prot -title pdb",shell=True, cwd="./PDB_blast_db")

### 2. Preliminary screen for proteins in the PDB database with homology to the random human fragment antigen-binding region 

Fetch PDB's with Fab's

In [ ]:
light = "DILLTQSPVILSVSPGERVSFSCRASQSIGTNIHWYQQRTNGSPRLLIKYASESISGIPSRFSGSGSGTDFTLSINSVESEDIADYYCQQNNNWPTTFGAGTKLELK"
with open("PDB_blast_db/fab_light.fasta",'w') as fo:
    fo.write(">input_light\n")
    fo.write(light)

heavy = "QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLEWLGVIWSGGNTDYNTPFTSRLSINKDNSKSQVFFKMNSLQSNDTAIYYCARALTYYDYEFAYWGQGTLVTVSA"
with open("PDB_blast_db/fab_heavy.fasta",'w') as fo:
    fo.write(">input_heavy\n")
    fo.write(heavy)

if not os.path.exists("PDB_blast_db/hits_fabs_light.xml"):
    subprocess.call("blastp -db pdb_seqres.txt -num_alignments 99999 -evalue 1e-9 -query fabs.fasta -out hits_fabs_light.xml -outfmt 5",shell=True, cwd="./PDB_blast_db")

if not os.path.exists("PDB_blast_db/hits_fabs_heavy.xml"):
    subprocess.call("blastp -db pdb_seqres.txt -num_alignments 99999 -evalue 1e-9 -query fabs.fasta -out hits_fabs_heavy.xml -outfmt 5",shell=True, cwd="./PDB_blast_db")



In [ ]:
def extractPDBIDsfromBlastOutput(input_path): 
    result=open(input_path,"r")
    records= NCBIXML.parse(result)
    item=next(records)
    pdb_fabs  = set()
    pdb_fabs_ = set()
    for alignment in item.alignments:
        #print(alignment)
        #break
        for hsp in alignment.hsps:
            pdb_id = alignment.title.split()[1]
            pdb_id_id = pdb_id.split("_")[0]
            pdb_fabs.add(pdb_id)
            pdb_fabs_.add(pdb_id_id)
    return pdb_fabs
pdb_fab_hits_1 =  extractPDBIDsfromBlastOutput("PDB_blast_db/hits_fabs_light.xml")
pdb_fab_hits_2 =  extractPDBIDsfromBlastOutput("PDB_blast_db/hits_fabs_heavy.xml")
pdb_fab_hits   = pdb_fab_hits_1|pdb_fab_hits_2

print(len(pdb_fab_hits))

### 3. Screen for heavy and light fab chains using ANARCI

http://opig.stats.ox.ac.uk/webapps/newsabdab/sabpred/anarci/ <br>
https://github.com/oxpig/ANARCI

Annotate all Fabs with ANARCI

In [ ]:
def loadFasta(path):
    r = []
    with open(path) as f:
        for line in f:
            if line[0]==">":
                r.append([])
            r[-1].append(line.rstrip())
    r = [[r_[0],"".join(r_[1:])] for r_ in r]
    return r

### Identify and annotate Fab's from the blast output results using ANARCI tool

In [ ]:
r = []
with open("./PDB_blast_db/pdb_seqres.txt") as f:
    for line in f:
        if line[0]==">":
            r.append([])
        r[-1].append(line)
        
pdb_seqres_fasta = loadFasta("./PDB_blast_db/pdb_seqres.txt")

rfabs = []
for r_ in r:
    title = r_[0].split(" ")[0][1:]
    if title not in pdb_fab_hits:
        continue
    rfabs.append([r_[0].split(" ")[0][1:],r_[1]])
    
with open("./PDB_blast_db/putative_fabs.fasta",'w') as fo:
    for r in rfabs:
        fo.write("".join([">"+r[0]+"\n",r[1]])+"\n")

if not os.path.exists("./PDB_blast_db/all_fabs_heavy.anarci"):
    anarci_command = "ANARCI -i putative_fabs.fasta -o all_fabs_heavy.anarci -s chothia -r ig --ncpu 8 --bit_score_threshold 100 --restrict heavy"
    subprocess.call(anarci_command,shell=True,cwd="./PDB_blast_db")
    
if not os.path.exists("./PDB_blast_db/all_fabs_light.anarci"):
    anarci_command = "ANARCI -i putative_fabs.fasta -o all_fabs_light.anarci -s chothia -r ig --ncpu 8 --bit_score_threshold 100 --restrict light"
    subprocess.call(anarci_command,shell=True,cwd="./PDB_blast_db")


In [ ]:
if not os.path.exists("./PDB_blast_db/all_fabs_heavy.anarci"):
    anarci_command = "/mnt/10tb/nvivanisenko/anaconda3/envs/folding/bin/ANARCI -i putative_fabs.fasta -o all_fabs_heavy.anarci -s chothia -r ig --ncpu 8 --bit_score_threshold 100 --restrict heavy"
    subprocess.call(anarci_command,shell=True,cwd="./PDB_blast_db")
    
if not os.path.exists("./PDB_blast_db/all_fabs_light.anarci"):
    anarci_command = "/mnt/10tb/nvivanisenko/anaconda3/envs/folding/bin/ANARCI -i putative_fabs.fasta -o all_fabs_light.anarci -s chothia -r ig --ncpu 8 --bit_score_threshold 100 --restrict light"
    subprocess.call(anarci_command,shell=True,cwd="./PDB_blast_db")

### 3.1 Parse ANARCI output and extract heavy and light Fab sequences 


In [ ]:
def parseAnarciAnnotation(path = "light.anarci",n=108):
    seqs = []
    seqs.append([[] for i in range(n)])
    used = set()
    data = {}
    with open(path) as f:
        w = f.readlines()
        data = [[]]
        for u,line in enumerate(w):
            data[-1].append(line)
            if line[0]=="/":
                data.append([])          
        out = {}
        for d in data:
            if len(d)==0:
                continue
            name = d[0].rstrip().split()[-1]
            if name in out:
                continue
            out[name] = [[] for i in range(n)]            
            for d_ in d:
                if d_[0]=="#":
                    continue
                if d_[0]=="/":
                    continue
                id_ = d_.split()[1]
                id_ = int(id_)
                if d_[10]=="-":
                    continue
                out[name][id_].append(d_[10])
    out_ = {}
    for name in out:
        if len("".join(["".join(c) for c in out[name]]))==0:
            continue
        out_[name] = out[name]
    return out_

anarci_list_heavy = parseAnarciAnnotation("./PDB_blast_db/all_fabs_heavy.anarci",n=120)
anarci_list_light = parseAnarciAnnotation("./PDB_blast_db/all_fabs_light.anarci",n=108)



### 4. Fetch all PDB structures containing Light/Heavy chains 

In [ ]:
pdb_3 = {r[:4]:{"light":[],"heavy":[]} for r in list(pdb_fabs)}

for h in anarci_list_light:
    h4 = h[:4]
    if h4 not in pdb_3:
        continue
    pdb_3[h4]["light"].append(h)
    
for h in anarci_list_heavy:
    h4 = h[:4]
    if h4 not in pdb_3:
        continue
    pdb_3[h4]["heavy"].append(h)
    
if not os.path.exists("PDB_blast_db/structs"):
    os.mkdir("PDB_blast_db/structs")

for pdb_ in pdb_3:
    if len(pdb_3[pdb_]["light"])+len(pdb_3[pdb_]["heavy"])==0:
        continue    
    pdb_name = pdb_.upper()+".pdb.gz"
    if os.path.exists("PDB_blast_db/structs/"+pdb_name):
        continue
    if os.path.exists("PDB_blast_db/structs/"+pdb_name.rstrip(".gz")):
        continue
    subprocess.call(f"wget https://files.rcsb.org/download/{pdb_name}",shell=True, cwd="./PDB_blast_db/structs")



### Keep only first model for multimodel PDB structures

In [ ]:
from pathlib import Path
pdbs = Path("./PDB_blast_db/structs/").glob("*.pdb")
for pdb in pdbs:
    print(pdb)
    pdb_data = pdb.open('r').readlines()
    fo = open(str(pdb),'w')#.open('w')
    for l in pdb_data:
        fo.write(l)
        if l.startswith("ENDMDL"):
            break
    fo.close()
    

### 5. Prepare PDB dataframes and align full sequence (from pdb seq-res) on sequence of resloved protein (may contain some gaps) 

In [ ]:
def getLetterFromResidueName(resn):
    d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
    return d[resn]

def kalign(seq1,seq2):
    if not os.path.exists("./PDB_blast_db/temp"):
        os.mkdir("./PDB_blast_db/temp")
    fo = open("./PDB_blast_db/temp/input.fasta",'w')
    fo.write(f">1\n{seq1}\n>2\n{seq2}\n")
    fo.close()
    d = subprocess.check_output("cat ./PDB_blast_db/temp/input.fasta | kalign -f fasta",shell=True)
    res_  = d.decode("UTF-8").split("\n")
    res   = []
    for l in res_:
        if len(l) ==0:
            continue
        if l[0]==">":
            res.append([])
        if len(res)!=0:
            res[-1].append(l.rstrip("\n"))
    return "".join(res[0][1:]),"".join(res[1][1:])
    
    
def removeAlternativeConformations(pdb_dataframe):
    return pdb_dataframe[(pdb_dataframe["alt_loc"] == "A") | (pdb_dataframe["alt_loc"] == " ")  | (pdb_dataframe["alt_loc"] == "")]
    
def removeUNK(pdb_dataframe):
    r = pdb_dataframe["residue_name"] == "UNK"
    return pdb_dataframe[~r]

def considerInsertions(pdb_dataframe):
    r1 = pdb_dataframe["residue_number"]
    r2 = pdb_dataframe["insertion"]
    r3 = pdb_dataframe["chain_id"]
    r4 = pdb_dataframe["residue_name"]
    ra = [(r_1,r_2,r_3,r_4) for (r_1,r_2,r_3,r_4) in zip(r1,r2,r3,r4)]
    pdb_dataframe["residue_key"] = ra
    return pdb_dataframe
    
def putFullSequence(pdb_dataframe, full_seq):
    pdb_dataframe = removeAlternativeConformations(pdb_dataframe)
    pdb_dataframe = removeUNK(pdb_dataframe)
    pdb_dataframe = considerInsertions(pdb_dataframe)
    
    if len(pdb_dataframe) == 0:
        print("Empty...")
        return
    pdb_ca                          = pdb_dataframe[pdb_dataframe["atom_name"] == "CA"]    
    residue_numbers = []
    residue_seq     = []
    used = set()
    
    #### To avoid residue duplicates 
    #### Might be not necessary
    for r in pdb_ca.iloc():#["residue_number"]:
        residue_number = r["residue_key"]
        if residue_number in used:
            continue
        residue_numbers.append(residue_number)
        residue_seq.append(    getLetterFromResidueName(r["residue_name"]))
    
    pdb_seq = "".join(residue_seq)#pdb_seq)    
    if len(pdb_seq) <= 5:
        print("PDB sequence is too short")
        return
    pdb_seq_aligned, full_seq_aligned = kalign(pdb_seq,full_seq)

    assert full_seq_aligned.replace("-","") == full_seq
    
    n_pdb     = -1
    n_pdb_map = []
    
    print("pdb",pdb_seq_aligned)
    print("full",full_seq_aligned)
    
    assert len(pdb_seq_aligned) == len(full_seq_aligned)
    
    new_dataframe = []
    for i,[a_pdb,a_fullseq] in enumerate(zip(list(pdb_seq_aligned),list(full_seq_aligned))):
        if a_pdb != '-':
            n_pdb+=1
        if a_pdb == "-":
            n_pdb_map.append({"resi":None,                  "a_pdb":None, "a_full":a_fullseq})
        else:
            n_pdb_map.append({"resi":residue_numbers[n_pdb],"a_pdb":a_pdb,"a_full":a_fullseq})

    full_df  = []
    full_seq = []
    pd.options.mode.chained_assignment = None  # default='warn'
    
    for r in n_pdb_map:
        full_seq.append(r["a_full"])
        
        if r["resi"] is None:
            empty_   = pd.DataFrame(np.nan, index=[0],columns=pdb_ca.columns)
            empty_["atom_name"] = "CA"
            empty_["seqres"]    = r["a_full"]
            full_df.append(empty_)
            continue
        
        pdb_residue = pdb_dataframe[pdb_dataframe["residue_key"] == r["resi"]]
        pdb_residue["seqres"] = r["a_full"]
        pdb_residue["aa"]     = r["a_pdb"]
        full_df.append(pdb_residue)  

    full_df = pd.concat(full_df,axis=0,ignore_index=True)
    return full_df

def getJobs():
    if not os.path.exists("./PDB_blast_db/all_pdbids_and_chains.txt"):
        all_chains = set()
        p = Path("./PDB_blast_db/structs/").glob("*.pdb")    
        for p_ in p:
            for line in p_.open('r'):
                if line.startswith("ATOM") and line[13:15] =="CA":
                    all_chains.add(p_.name.rstrip(".pdb")+"_"+line[21])                
        with open("./PDB_blast_db/all_pdbids_and_chains.txt",'w') as fo:
            fo.write("\n".join(list(all_chains)))

    all_chains = [r.rstrip() for r in open("./PDB_blast_db/all_pdbids_and_chains.txt",'r').readlines()]
    used = set()
    for p in Path("./PDB_blast_db/structs_per_chain/").glob("*.pkl"):
        used.add(p.name.rstrip(".pkl"))
    j = {}
    for u in all_chains:
        if u in used:
            continue
        pdbid,chain = u.split("_")
        j.setdefault(pdbid,set())
        j[pdbid].add(chain)
    return j



def getPDBDataFrame(pdb_id = "1FGV",chains = None):   
    pdb_path           = f"./PDB_blast_db/structs/{pdb_id.upper()}.pdb"
    if not os.path.exists(pdb_path):
        print(pdb_id," not found")
        return
    pdb_structure      = PandasPdb().read_pdb(pdb_path).df["ATOM"]
    
    from Bio import SeqIO
    
    sequences = {}
    pdb_records = {}
    
    if not os.path.exists(f"./PDB_blast_db/structs_per_chain/"):
        os.mkdir(f"./PDB_blast_db/structs_per_chain/")

    for record in SeqIO.parse(pdb_path, "pdb-seqres"):
        chain             = record.id[-1]            
        if chains is not None and chain not in chains:
            continue
        sequences[chain]  = record.seq
        pdb_chain         = pdb_structure[pdb_structure["chain_id"] == chain]
        if len(pdb_chain) == 0:
            continue        
        print(pdb_id,chain)
        pdb_chain_fullseq = putFullSequence(pdb_chain,sequences[chain])
        pickle.dump(pdb_chain_fullseq, open(f"./PDB_blast_db/structs_per_chain/{pdb_id}_{chain}.pkl",'wb'))

jobs = getJobs()

for pdb_id in getJobs():
    chains = jobs[pdb_id]
    getPDBDataFrame(pdb_id,chains)
    




### 6. Put ANARCI annotation into antibodies dataframes prepared in the previous step

In [ ]:
def mafft_align(s1,s2,strict=True):
    with open("m.fasta",'w') as fo:
        fo.write(">1\n"+s1+"\n>2\n"+s2+"\n")
    if not strict:
        d = subprocess.check_output("mafft --anysymbol --op 0.1  m.fasta ",shell=True)
    else:
        d = subprocess.check_output("mafft --anysymbol --auto m.fasta ",shell=True)

    res_  = d.decode("UTF-8").split("\n")
    res   = []
    for l in res_:
        if len(l)==0:
            continue
        if l[0]==">":
            res.append("")
            continue
        res[-1]+=l.rstrip()
    return res

def realignSequences(pdb_seq,anarci_, firstLetterException = False):
    seq_aa = []
    seq_i  = []
    for i,s_ in enumerate(anarci_):
        if len(s_)==0:
            continue
        seq_aa+=s_
        seq_i +=[i for i_ in range(len(s_))]
    al = kalign("".join(seq_aa),"".join(pdb_seq))
    seq_anarci_aligned = al[0]
    pdb_seq_aligned    = al[1]
    n_anarci = 0
    n_pdb    = 0
    pdb_anarci_map = [None for i in pdb_seq] 
    
    for i,[a_anarci,a_pdb] in enumerate(zip(*al)):
        if a_anarci!="-" and a_pdb!="-":
            pdb_anarci_map[n_pdb] = i
            if n_anarci == 0 and firstLetterException:
                n_pdb+=1
                n_anarci+=1
                continue
            if a_pdb!=a_anarci:
                return None
        if a_pdb!="-":
            n_pdb+=1
        if a_anarci!="-":            
            n_anarci+=1
            
    return pdb_anarci_map
    
def putAnarciAnnotation(pdb_dataframe,fab_id, firstLetterException = False):
    pdb_id,chain,fab_type = fab_id
    if fab_type == "light":
        anarci_seq     = anarci_list_light[pdb_id.lower()+"_"+chain]
    else:
        anarci_seq     = anarci_list_heavy[pdb_id.lower()+"_"+chain]
    pdb_ca        = pdb_dataframe[pdb_dataframe["atom_name"] == "CA"]
    pdb_seq = "".join(pdb_ca["seqres"])
    pdb_anarci_map = realignSequences(pdb_seq, anarci_seq,firstLetterException)
    if pdb_anarci_map is None:
        return None
    pdb_anarci_map = [fab_type[0].upper()+str(i)  if i is not None else None for i in pdb_anarci_map]    
    pdb_dataframe["anarci"] = None
    for anarci_id,residue_number in zip(pdb_anarci_map, pdb_ca["residue_key"]):
        ids = pdb_dataframe[ "residue_key"] == residue_number
        pdb_dataframe.loc[ids,"anarci"] = anarci_id     
    return pdb_dataframe





###  An example of annotated Fab

In [ ]:
pdb_id, chain, fab_type = ("7LYW","H","heavy")
pdb_id, chain, fab_type = ("5HCG","H","heavy")
pdb_path           = f"./PDB_blast_db/structs_per_chain/{pdb_id}_{chain}.pkl"
out_path           = f"./PDB_blast_db/structs_antibodies/{pdb_id}_{chain}_{fab_type}.pkl"
fab                = pickle.load(open(pdb_path,'rb'))
fab_annotated      = putAnarciAnnotation(fab, (pdb_id,chain,fab_type), firstLetterException=True)
print(fab_annotated)

In [ ]:
if not os.path.exists(f"./PDB_blast_db/structs_antibodies/"):
    os.mkdir(f"./PDB_blast_db/structs_antibodies/")
    
    
jobs = []
for anarci_id,anarci_map in anarci_list_heavy.items():    
    jobs.append((anarci_id[:4].upper(),anarci_id[-1],"heavy"))
for anarci_id,anarci_map in anarci_list_light.items():    
    jobs.append((anarci_id[:4].upper(),anarci_id[-1],"light"))

strange_error_list = set()

for pdb_id,chain,fab_type in jobs:
    if pdb_id.upper()+"_"+chain in strange_error_list:
        continue        
    firstLetterException = True
    print(pdb_id,chain,fab_type)

    pdb_path           = f"./PDB_blast_db/structs_per_chain/{pdb_id}_{chain}.pkl"
    if not os.path.exists(pdb_path):
        continue
        
    out_path           = f"./PDB_blast_db/structs_antibodies/{pdb_id}_{chain}_{fab_type}.pkl"
    if os.path.exists(out_path):
        continue
    fab                = pickle.load(open(pdb_path,'rb'))
    fab_annotated      = putAnarciAnnotation(fab, (pdb_id,chain,fab_type),firstLetterException)
    if fab_annotated is None:
        strange_error_list.add((pdb_id,chain,fab_type))
        continue            
    pickle.dump(fab_annotated, open(out_path,'wb'))
    

### Following structures got troubles in annotation and were ignored later 

In [ ]:
for e in strange_error_list:
    print(e)    

### 7. Identify Fab pairing

In [ ]:

def getPDBList():
    return [p.name[:4] for p in Path("./PDB_blast_db/structs_antibodies/").glob(f"*.pkl")]
    
def getFabsPDBID(pdb_id = "1LK3"):
    fab_path = Path("./PDB_blast_db/structs_antibodies/").glob(f"{pdb_id}*.pkl")
    fab_ids  = {"heavy":[],"light":[]}
    for struct_id in fab_path:
        pdb_id, chain, fab_type = struct_id.name.rstrip(".pkl").split("_")
        fab_ids[fab_type].append(struct_id)
    return fab_ids


In [ ]:
import scipy
from scipy.spatial import distance
def getxyz(df):
    xyz = np.array([df["x_coord"],df["y_coord"],df["z_coord"]]).T
    return xyz
    
    
def getPairInterface(path_light, path_heavy):
    
    pdb_light = pickle.load(open(path_light,'rb'))
    pdb_heavy = pickle.load(open(path_heavy,'rb'))
    
    heavy_interface = list(range(32,39)) + list(range(44,50)) + list(range(85,95))
    light_interface = list(range(34,39)) + list(range(45,51)) + list(range(90,108))

    heavy_ids = ["H"+str(i) for i in heavy_interface]
    light_ids = ["L"+str(i) for i in heavy_interface]
    
    heavy_interface = pdb_heavy[pdb_heavy["anarci"].isin(heavy_ids)]
    light_interface = pdb_light[pdb_light["anarci"].isin(light_ids)]
    
    xyz_heavy = getxyz(heavy_interface)
    xyz_light = getxyz(light_interface)
    cd = distance.cdist(xyz_heavy,xyz_light)
    ids = np.where(cd<4.5)
    
    return len(set(ids[0]))+len(set(ids[1]))
    
def screenFabPairs(pdb_id):
    fab_path = getFabsPDBID(pdb_id)
    contacts = {}
    for heavy_path in fab_path["heavy"]:
        for light_path in fab_path["light"]:
            n = getPairInterface(path_light = light_path, path_heavy = heavy_path)
            if n > 3:
                contacts[(light_path.name.rstrip(".pkl"),heavy_path.name.rstrip(".pkl"))] = n
    return contacts

fab_contacts = {}
for pdb_id in getPDBList():
    fab_contacts[pdb_id] = screenFabPairs(pdb_id)
    print(pdb_id,fab_contacts[pdb_id])
        

In [ ]:
pickle.dump(fab_contacts, open("./PDB_blast_db/fab_pairs.pkl",'wb'))

### 8. Find antigens and interacting antibodies

In [ ]:
def getAllFabs():
    return [l+".pkl" for l in fab_ids]

def getAllAntigensList():
    fab_ids = set(anarci_list_heavy)|set(anarci_list_light)
    fab_ids = {f[:4].upper()+"_"+f[-1] for f in fab_ids}#print(fab_ids)
    pdb_ids = [p for p in Path("./PDB_blast_db/structs_per_chain/").glob("*.pkl") if p.name[:6] not in fab_ids]
    return pdb_ids

def getAntigensPDBID(pdb_id = "1LK3"):
    all_antigens = getAllAntigensList()
    return [a for a in all_antigens if a.name[:4] == pdb_id]


def screenAntigenContacts(pdb_id = "1LK3"):
    if pdb_id not in fab_contacts:
        return []
    fab_pairs = fab_contacts[pdb_id]
    antigens  = getAntigensPDBID(pdb_id)
    hits = []
    for antigen in antigens:
        for fab_pair in fab_pairs:
            fab_path_light = Path("./PDB_blast_db/structs_antibodies/"+fab_pair[0]+".pkl")
            fab_path_heavy = Path("./PDB_blast_db/structs_antibodies/"+fab_pair[1]+".pkl")
            n_light = testContact(antigen,fab_path_light)
            n_heavy = testContact(antigen,fab_path_heavy)
            if n_light+n_heavy == 0:
                continue
            hits.append([antigen,fab_pair,n_light,n_heavy])
    return hits
        
def testContact(antigen_path, fab_path):
    print(antigen_path)
    antigen_df = pickle.load(antigen_path.open('rb'))
    if antigen_df is None:
        return 0

    fab_df     = pickle.load(fab_path.open('rb'))
    fab_type   = None
    interface  = None
    
    if fab_path.name.split("_")[-1] == "light.pkl":
        fab_type  = "light"
        interface = ["L"+str(i) for i in list(range(23,35))+list(range(66,72))+list(range(89,98))]
        
    elif fab_path.name.split("_")[-1] == "heavy.pkl":
        fab_type  = "heavy" 
        interface = ["H"+str(i) for i in list(range(23,35))+list(range(51,57))+list(range(93,102))]
    
    fab_interface = fab_df[fab_df["anarci"].isin(interface)]
    xyz_fab       = getxyz(fab_interface)
    xyz_antigen   = getxyz(antigen_df)                   
    
    cd  = distance.cdist(xyz_antigen,xyz_fab)
    ids = set(np.where(cd<4.5)[0])
    
    return len(ids)

    
hits = []
for pdb_id in set([n.name[:4] for n in getAllAntigensList()]):
    hits+=screenAntigenContacts(pdb_id)
    
fo = open("./PDB_blast_db/antigen_fabs.txt",'w')
for r in hits:
    fo.write(f"{r[0]} {r[1][0]} {r[1][1]} {r[2]} {r[3]}\n")
fo.close()


### 8. We have prepared dataframes and PDB structures including heavy / light fab pairs interacting with antigen.
### Contact numbers are going to be calculated with PyMOL script contact_number.py <br>
Here is an example for R1 = 4.5 and R2 = 12.0 <br>


In [ ]:
R1 = 4.5
R2 = 12.0 
for fab_antigen_id in fab_antigen_ids:
    cmd.do(f"pymol -r contact_number.py {fab_antigen_id} {R1} {R2}")

### 9. Cluster sequences , extract release dates , and combine all contact numbers to single antigen-epitope dataframe

### Useful scripts for sequence clustering and dataset generation

In [ ]:
import os
import pickle
from   pathlib import Path
import pandas as pd
import subprocess
import numpy as np


def alignmentLabels(ref_seq,tar_seq):
    """
    Function to align residue position from two prealigned sequences ref_seq and tar_seq
    """
    n  = [None for i in range(len(ref_seq.replace("-","")))]
    n1 = 0
    n2 = 0
    for s1,s2 in zip(ref_seq,tar_seq):
        if s1!="-" and s2!="-":
            n[n1] = n2
        if s1!="-":
            n1+=1
        if s2!="-":
            n2+=1
    return n

#    

def loadMSA(path,dataframe):
    """
    Function to parse MSA
    """
    seqs = []
    for line in path.open('r'):
        if line[0]==">":
            seqs.append([])
        seqs[-1].append(line.rstrip())

    sequences = []
    titles    = []
    for s in seqs:
        title = s[0][1:]
        if title not in dataframe:
            continue
        sequences.append("".join(s[1:]))
        titles.append(s[0][1:])

    sequences_resi = []
    seq_ids = {}
    msa_matrix = np.zeros((len(sequences),len(sequences[0]))).astype(int)-1

    for u,[sequence,title] in enumerate(zip(sequences,titles)):
        seq_ids[title] = []
        n = 0
        for i in range(len(sequence)):
            if sequence[i] == "-":
                continue
            msa_matrix[u,i] = n
            seq_ids[title].append(i)
            n+=1
    return msa_matrix, sequences, titles, seq_ids
    
def calculateConsensus(data, job_id = "_fab"):
    """
    precalculate MSA for 95% identity clusters of antigen sequences
    mmseqs is used to for clustering
    kalign for MSA
    """
    if not os.path.exists(f"./temp{job_id}/"):
        os.mkdir(f"temp{job_id}/")
    
    fo = open(f"./temp{job_id}/all_sequences.fasta",'w')
    sequences = {}
    for pdbid in data:
        seq = "".join(data[pdbid]["aa"])
        if len(seq) < 50:
            continue
        fo.write(f">{pdbid}\n{seq}\n")
        sequences[pdbid] = seq
        print(pdbid,seq)
    fo.close()
        
    command = """mmseqs easy-cluster ../all_sequences.fasta results temp --min-seq-id 0.95 -c 0.9 --cov-mode 0 -s 8 --cluster-mode 1"""
    
    if not os.path.exists(f"./temp{job_id}/clusters_095"):
        os.mkdir(f"./temp{job_id}/clusters_095")
    
   
    clusters = {}
    with open(f"./temp{job_id}/clusters_095/results_cluster.tsv") as f:
        for line in f:
            r = line.rstrip().split()
            clusters.setdefault(r[0],set())
            clusters[r[0]].add(r[1])
    
    if not os.path.exists(f"./temp{job_id}/clusters_095/msa/"):
        os.mkdir(f"./temp{job_id}/clusters_095/msa")

    fo_jobs = open(f"./temp{job_id}/msa_095.sh",'w')    
    for cluster_ref in clusters:    
        fo = open(f"./temp{job_id}/clusters_095/msa/{cluster_ref}.fasta",'w')
        fo.write(f">{cluster_ref}\n{sequences[cluster_ref]}\n")
        for c in clusters[cluster_ref]:
            if c == cluster_ref:
                continue
            fo.write(f">{c}\n{sequences[c]}\n")
        fo.close()
        fo_jobs.write(f"kalign -i ./clusters_095/msa/{cluster_ref}.fasta -o ./clusters_095/msa/{cluster_ref}_al.fasta\n")
    fo_jobs.close()

def consensusByMsa(dataframe,output_folder):
    """
    parse precalculate MSA and calculate consensus contact numbers
    """
    msa_path = Path("./temp_fab/clusters_095/msa/").glob("*_al.fasta")
    for m in msa_path:
        consensusByMsa_(dataframe,
                        msa_path = str(m),
                        output_path = output_folder)
    
def consensusByMsa_(dataframe,
                    msa_path = "./temp_fab/clusters_095/msa/7T0R_A_LH_al.fasta",
                    output_path = "./temp_fab/clusters_095/pkls/"):
    """
    consensus contact numbers are calculated for antigens within 95% identity clusters 
    """
    
    
    output_full_path = output_path+"/"+msa_path.split("/")[-1]+".pkl"

    if os.path.exists(output_full_path):
        return
    
    msa, msa_seq, msa_name, msa_ids = loadMSA(Path(msa_path),dataframe)
    
    values = {"cn_CA":{},"cn_raw":{},"cn_non_raw":{}}
    for seq_id in range(len(msa_name)):
        for msa_id in range(msa.shape[1]):
            resi_id = msa[seq_id,msa_id]
            if resi_id == -1:
                continue
            aa = msa_seq[seq_id][msa_id]
            for c in ["cn_CA","cn_raw","cn_non_raw"]:
                values[c].setdefault(msa_id,{})
                values[c][msa_id].setdefault(aa,[])
                contacts = dataframe[msa_name[seq_id]][c].iloc[resi_id]
                values[c][msa_id][aa].append(contacts)
                
    for contact_id in values:
        for msa_id in values[contact_id]:
            for aa in values[contact_id][msa_id]:
                values[contact_id][msa_id][aa] = max(values[contact_id][msa_id][aa])

    cluster = {}
    for pdbid in msa_ids:
        consensus = {"cn_CA":[],"cn_raw":[],"cn_non_raw":[]}
        for resi_id,msa_id in enumerate(msa_ids[pdbid]):
            aa = dataframe[pdbid].iloc[resi_id]["aa"]
            assert aa in values[c][msa_id]
            for c in values:
                consensus[c].append(values[c][msa_id][aa])
                
        for c in consensus:
            dataframe[pdbid][c+"_consensus"] = consensus[c]
            
        cluster[pdbid] = dataframe[pdbid]

    if not os.path.exists(output_path):
        os.mkdir(output_path)
    
    pickle.dump(cluster,open(output_full_path,'wb'))

    return        
        

def combineDataframes(input_ = "./structs_antigen_fab_df/" ,
                      output_ = "./true_antigen_fab_cn_12.0_6.0.pkl",
                      R = "12.0_6.0"):
    """
    combine dataframes
    """
    r = Path(input_).glob(f"*_{R}.pkl")
    contact_numbers = {}    
    for r_ in r:
        df = pickle.load(r_.open('rb'))
        df = pd.DataFrame(df)
        df["name"] = r_.name.rstrip(".pkl")
        pdb_name = r_.name.split("_")[:3]
        pdb_name = "_".join(pdb_name)
        contact_numbers[pdb_name] = df
    pickle.dump(contact_numbers, open(output_,'wb'))

    
def verySimilarClusters(data, job_id = "_all"):
    """
    cluster sequences by 95% identity
    """

    if not os.path.exists(f"./temp{job_id}/"):
        os.mkdir(f"./temp{job_id}/")
        
    if not os.path.exists(f"./temp{job_id}/clusters_095"):
        os.mkdir(f"./temp{job_id}/clusters_095")
        
    fo = open(f"./temp{job_id}/clusters_095/all_sequences.fasta",'w')
    sequences = {}
    for pdbid in data:
        seq = "".join(data[pdbid]["aa"])
        if len(seq) < 50:
            continue
        fo.write(f">{pdbid}\n{seq}\n")
        sequences[pdbid] = seq
        print(pdbid,seq)
    fo.close()
    
    command = """mmseqs easy-cluster all_sequences.fasta results temp --min-seq-id 0.95 -c 0.9 --cov-mode 0 -s 8 --cluster-mode 1"""

    subprocess.call(command,shell=True,cwd = f"./temp{job_id}/clusters_095/")

    clusters = {}
    with open(f"./temp{job_id}/clusters_095/results_cluster.tsv") as f:
        for line in f:
            r = line.rstrip().split()
            clusters.setdefault(r[0],set())
            clusters[r[0]].add(r[1])

    clu_id = 0
    clusters_ = {}
    for c in clusters:
        for pdb_id in clusters[c]:
            clusters_[pdb_id] = clu_id
        clu_id +=1

    for pdb_id in sequences:
        print(pdb_id)
        assert pdb_id in clusters_
        data[pdb_id]["very_similar_cluster_id"] = clusters_[pdb_id]
    
    return clusters_

def notverySimilarClusters(data, job_id = "_all"):
    """
    cluster sequences by 70% identity
    """
    if not os.path.exists(f"./temp{job_id}/"):
        os.mkdir(f"./temp{job_id}/")
        
    if not os.path.exists(f"./temp{job_id}/clusters_07"):
        os.mkdir(f"./temp{job_id}/clusters_07")

    fo = open(f"./temp{job_id}/clusters_07/all_sequences.fasta",'w')
    sequences = {}
    for pdbid in data:
        seq = "".join(data[pdbid]["aa"])
        if len(seq) < 50:
            continue
        fo.write(f">{pdbid}\n{seq}\n")
        sequences[pdbid] = seq
        print(pdbid,seq)
    fo.close()

    command = f"""mmseqs easy-cluster all_sequences.fasta results temp --min-seq-id 0.7 -c 0.7 --cov-mode 1 -s 8 --cluster-mode 1"""

    subprocess.call(command,shell=True,cwd = f"./temp{job_id}/clusters_07/")

    clusters = {}
    with open(f"./temp{job_id}/clusters_07/results_cluster.tsv") as f:
        for line in f:
            r = line.rstrip().split()
            clusters.setdefault(r[0],set())
            clusters[r[0]].add(r[1])

    clu_id = 0
    clusters_ = {}
    for c in clusters:
        for pdb_id in clusters[c]:
            clusters_[pdb_id] = clu_id
        clu_id +=1

    for pdb_id in sequences:
        print(pdb_id)
        assert pdb_id in clusters_
        data[pdb_id]["notvery_similar_cluster_id"] = clusters_[pdb_id]
    
    return clusters_
 


def combineAllData(exp_type = "_12.0_4.5"):
    if not os.path.exists(f"all_antigen_noclusters{exp_type}.pkl"):
        df_fab = {}
        df_msa_path = Path(f"./msa{exp_type}/").glob("*.pkl")
        clu_id = 0
        
        for d in df_msa_path:
            dfs = pickle.load(d.open('rb'))
            
            for pdb_id in dfs:
                dfs[pdb_id]["cluster_id"] = clu_id
            df_fab.update(dfs)

            clu_id+=1
            print(clu_id,len(df_fab))
         
        df1  = pickle.load(open(f"./true_antigen_fab_cn{exp_type}.pkl",'rb'))
        
        print(len(df_fab),"some missing?")
        print(len(df_fab),"before")
        
        for pdb_id in df1:
            if df1[pdb_id].shape[0]<50:
                continue
            if pdb_id not in df_fab:
                df_fab[pdb_id] = df1[pdb_id]

        print(len(df_fab),"after")
        print("now add missing fake antigens")
         
        pickle.dump(df_fab,open(f"all_antigen_noclusters{exp_type}.pkl",'wb'))
    else:
        df_all = pickle.load(open(f"all_antigen_noclusters{exp_type}.pkl",'rb'))

    print("now cluster very similar")
    verySimilarClusters(df_all,job_id = exp_type)

    print("now cluster not very similar")
    notverySimilarClusters( df_all, job_id = exp_type)

    pickle.dump(df_all, open(f"all_antigen_data{exp_type}.pkl",'wb'))
    
        
def putDates():
    """
    pdb_release_data.txt file is download from rcsb.org 
    function add release dates column for each antigen sequence cluster
    """
    date = {}
    with open("pdb_release_date.txt") as f:
        for line in f:
            r = line.rstrip().split()
            
            d = int(r[1].split("-")[-1])
            if d>=23:
                d = 1900+d
            else:
                d = 2000+d
            date[r[0]] = d

    data = pickle.load(open("all_antigen_data.pkl",'rb'))
    
    cluster_antigen_release_dates    = {}
    cluster_notantigen_release_dates = {}

    for pdb_id in data:
        if len(pdb_id)!=9:
            continue
        pdb4 = pdb_id.split("_")[0].upper()
        cluster_id = data[pdb_id]["notvery_similar_cluster_id"][0]
        cluster_antigen_release_dates.setdefault(cluster_id, [])
        cluster_antigen_release_dates[cluster_id].append(date[pdb4])
        
    for c in cluster_antigen_release_dates:
        cluster_antigen_release_dates[c] = min(cluster_antigen_release_dates[c])

    print(cluster_antigen_release_dates)
    cluster_notantigen_release_dates = {}
    for pdb_id in data:
        if len(pdb_id)!=8:
            continue
        pdb4 = pdb_id.split("_")[0].upper()
        cluster_id = data[pdb_id]["notvery_similar_cluster_id"][0]
        cluster_notantigen_release_dates.setdefault(cluster_id, [])
        cluster_notantigen_release_dates[cluster_id].append(date[pdb4])

    print(cluster_notantigen_release_dates)

    for c in cluster_notantigen_release_dates:
        cluster_notantigen_release_dates[c] = min(cluster_notantigen_release_dates[c])
    
        
    for pdb_id in data:
        pdb4 = pdb_id.split("_")[0].upper()
        cluster_id = data[pdb_id]["notvery_similar_cluster_id"][0]
        if len(pdb_id) == 9:
            cluster_release_date = cluster_antigen_release_dates[cluster_id] 
        else:
            cluster_release_date = cluster_notantigen_release_dates[cluster_id] 
        data[pdb_id]["release_date"] = date[pdb4]
        data[pdb_id]["cluster_release_date"] = cluster_release_date

    pickle.dump(data, open("all_antigen_data_date.pkl",'wb'))





In [ ]:
def pipeline(R1 = 6.0, R2 = 12.0):
    ### first combine all calculated contact numbers by PyMOL
    epitope_labels = f"./eptiope_labels_cn_{R1}_{R2}.pkl"
    
    if not os.path.exists(epitope_labels):
        combineDataframes(input_ = "./structs_antigen_fab_df/",
                          output_ = epitope_labels,
                          R = f"{R1}_{R2}")
    df1  = pickle.load(open(epitope_labels,'rb'))
        
    ### cluster sequences by similarity and calculate MSA for each cluster
    calculateConsensus(df1)
    
    ### get consensus contact number values according to MSA
    consensusByMsa(df1,
                   output_folder = f"./msa_{R1}_{R2}/")

    ### combine all data to single dataframe
    all_data_path = f"all_antigen_data_{R1}_{R2}.pkl"
    if not os.path.exists(all_data_path):
        combineAllData(f"_{R1}_{R2}")


    

### Example to generate dataset for R1 = 4.5 and R2 = 16.0

In [ ]:
pipeline(R1 = 4.5, R2 = 16.0)